In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-16 23:26:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2021-07-16 23:26:36 (6.02 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving allstations_cleaned.csv to allstations_cleaned.csv


In [ ]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['allstations_cleaned.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
# Sort by total docks column to find a busy station with plenty of dock for further exploration 
# (see Citibike_station422 notebook for analysis)
df2.sort_values(by = 'tot_docks')

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key
1986911,3466,W 45 St & 6 Ave,19-04-29,9,35,0,0,0,0,40.756687,-73.982577,0,3
261132,465,Broadway & W 41 St,16-05-01,5,31,1,0,0,0,40.755136,-73.986580,0,3
261133,465,Broadway & W 41 St,16-05-01,5,45,1,0,0,0,40.755136,-73.986580,0,3
261134,465,Broadway & W 41 St,16-05-01,5,59,1,0,0,0,40.755136,-73.986580,0,3
261135,465,Broadway & W 41 St,16-05-01,6,12,1,0,0,0,40.755136,-73.986580,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698808,501,FDR Drive & E 35 St,19-04-06,10,44,1,31,43,79,40.744219,-73.971212,1,1
1698807,501,FDR Drive & E 35 St,19-04-06,10,0,1,30,44,79,40.744219,-73.971212,1,1
1698820,501,FDR Drive & E 35 St,19-04-07,6,52,0,40,38,79,40.744219,-73.971212,1,1
1653195,422,W 59 St & 10 Ave,19-04-18,10,53,0,3,74,79,40.770513,-73.988038,1,1


In [ ]:
df2.groupby(['dock_name']).mean().sort_values(by = 'avail_bikes').loc['W 59 St & 10 Ave']

dock_id        422.000000
hour             6.459988
minute          29.448857
pm               0.485560
avail_bikes      9.038207
avail_docks     50.502407
tot_docks       60.516245
_lat            40.770513
_long          -73.988038
in_service       1.000000
status_key       1.000000
Name: W 59 St & 10 Ave, dtype: float64

In [ ]:
df2.nunique()

dock_id        759
dock_name      770
date            94
hour            12
minute          60
pm               2
avail_bikes     79
avail_docks     80
tot_docks       75
_lat           776
_long          775
in_service       2
status_key       2
dtype: int64

In [ ]:
#  Convert date column to datetime from object
df2['date'] = pd.to_datetime(df2['date'], yearfirst=True)
df2

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key
0,72,W 52 St & 11 Ave,2016-05-01,1,9,0,11,25,39,40.767272,-73.993929,1,1
1,72,W 52 St & 11 Ave,2016-05-01,1,22,0,11,25,39,40.767272,-73.993929,1,1
2,72,W 52 St & 11 Ave,2016-05-01,1,35,0,11,25,39,40.767272,-73.993929,1,1
3,72,W 52 St & 11 Ave,2016-05-01,1,47,0,11,25,39,40.767272,-73.993929,1,1
4,72,W 52 St & 11 Ave,2016-05-01,2,0,0,11,25,39,40.767272,-73.993929,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028341,3537,Carlton Ave & Dean St,2019-04-23,7,32,0,6,23,29,40.680974,-73.971010,1,1
2028342,3537,Carlton Ave & Dean St,2019-04-23,8,21,0,4,25,29,40.680974,-73.971010,1,1
2028343,3537,Carlton Ave & Dean St,2019-04-23,9,10,0,1,28,29,40.680974,-73.971010,1,1
2028344,3537,Carlton Ave & Dean St,2019-04-23,12,44,0,8,21,29,40.680974,-73.971010,1,1


In [ ]:
df2.dtypes

dock_id                 int64
dock_name              object
date           datetime64[ns]
hour                    int64
minute                  int64
pm                      int64
avail_bikes             int64
avail_docks             int64
tot_docks               int64
_lat                  float64
_long                 float64
in_service              int64
status_key              int64
dtype: object

In [ ]:
# Split date column into 'date', 'month' and 'year' columns
df2['day'] = df2['date'].dt.day
df2['month'] = df2['date'].dt.month
df2['year'] = df2['date'].dt.year
df2

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,day,month,year
0,72,W 52 St & 11 Ave,2016-05-01,1,9,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
1,72,W 52 St & 11 Ave,2016-05-01,1,22,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
2,72,W 52 St & 11 Ave,2016-05-01,1,35,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
3,72,W 52 St & 11 Ave,2016-05-01,1,47,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
4,72,W 52 St & 11 Ave,2016-05-01,2,0,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028341,3537,Carlton Ave & Dean St,2019-04-23,7,32,0,6,23,29,40.680974,-73.971010,1,1,23,4,2019
2028342,3537,Carlton Ave & Dean St,2019-04-23,8,21,0,4,25,29,40.680974,-73.971010,1,1,23,4,2019
2028343,3537,Carlton Ave & Dean St,2019-04-23,9,10,0,1,28,29,40.680974,-73.971010,1,1,23,4,2019
2028344,3537,Carlton Ave & Dean St,2019-04-23,12,44,0,8,21,29,40.680974,-73.971010,1,1,23,4,2019


In [ ]:
# Drop the date column
df2.drop(columns=["date"], inplace=True)
df2.head()

,dock_id,dock_name,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,day,month,year
0,72,W 52 St & 11 Ave,1,9,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
1,72,W 52 St & 11 Ave,1,22,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
2,72,W 52 St & 11 Ave,1,35,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
3,72,W 52 St & 11 Ave,1,47,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016
4,72,W 52 St & 11 Ave,2,0,0,11,25,39,40.767272,-73.993929,1,1,1,5,2016


In [ ]:
df3 = df2[['dock_id', 'dock_name']].copy()
df3

,dock_id,dock_name
0,72,W 52 St & 11 Ave
1,72,W 52 St & 11 Ave
2,72,W 52 St & 11 Ave
3,72,W 52 St & 11 Ave
4,72,W 52 St & 11 Ave
...,...,...
2028341,3537,Carlton Ave & Dean St
2028342,3537,Carlton Ave & Dean St
2028343,3537,Carlton Ave & Dean St
2028344,3537,Carlton Ave & Dean St


In [ ]:
range(0,len(df3))

range(0, 2028346)